In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import pandas as pd
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from loaddata import load_data

c:\users\youss\pycharmprojects\age-extraction\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\youss\pycharmprojects\age-extraction\venv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\youss\pycharmprojects\age-extraction\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data, label = load_data()

In [9]:
train_data =  np.array(data[:49255])
train_labels = np.array(label[:49255])
test_data = np.array(data[49255:])
test_labels = np.array(label[49255:])

In [ ]:
learning_rate = 0.001
epochs = 1
batch_size = 32

In [11]:
def create_model(learning_rate):
    model = models.Sequential()

    # Define the first hidden layer with 20 nodes.
    # model.add(tf.keras.layers.Dense(units=10,
    #                                 activation='softmax',
    #                                 name='Hidden1',
    #                                 activity_regularizer=regularizers.l2(0.01)
    #                                 ))

    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=(100, 100, 3)))
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.2))

    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))



    # model.add(layers.Conv2D(128, (3, 3), padding='same'))
    # model.add(layers.Activation('relu'))
    # model.add(layers.Dropout(0.2))
    # model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))


    model.add(layers.Dense(128, kernel_constraint=maxnorm(3)))
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(1))
    model.add(layers.Activation('softmax'))

    # model.add(layers.Dense(10))

    # checkpoint = ModelCheckpoint('best_model_improved.h5',
    #                              monitor='val_loss',
    #                              verbose=0,
    #                              save_best_only=True,
    #                              mode='auto')
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate),
                  loss="mean_squared_error",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

    return model


def train_model(model, train_images, train_labels, test_images, test_labels, epochs, batch_size=None):
    # Split the dataset into features and label.

    history = model.fit(train_images, train_labels, batch_size=batch_size,
                        epochs=epochs, shuffle=True, validation_data=(test_images, test_labels))

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # To track the progression of training, gather a snapshot
    # of the model's mean squared error at each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist


def plot_curve(epochs, hist):
    """Plot a curve of one or more classification metrics vs. epoch."""
    # list_of_metrics should be one of the names shown in:
    # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    # for m in list_of_metrics:
    x = hist["accuracy"]
    plt.plot(epochs[1:], x[1:], label="accuracy")

    plt.show()

In [12]:
my_model = create_model(learning_rate)

In [ ]:
epochs, hist = train_model(my_model, train_data, train_labels, test_data, test_labels, epochs, batch_size)